In [2]:
from dotenv import load_dotenv
load_dotenv()
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import os
import json


In [3]:
from pinecone import Pinecone, ServerlessSpec

pc=Pinecone(api_key="9ef71535-c0e4-4571-aa9e-29c59fda421b")
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)



PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '77477b36c47f0cb53c1d672349c789a8', 'Date': 'Fri, 23 Aug 2024 16:28:09 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [4]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Carter',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Dr. Carter is an excellent professor who explains complex concepts clearly. Her lectures are engaging and she is always willing to help students.'},
 {'professor': 'Dr. Robert Jones',
  'subject': 'Psychology',
  'stars': 4,
  'review': "Dr. Jones is knowledgeable and passionate about psychology. His exams are tough but fair. Highly recommended if you're willing to put in the work."},
 {'professor': 'Dr. Lisa Patel',
  'subject': 'Mathematics',
  'stars': 3,
  'review': 'Dr. Patel is good at teaching but her assignments can be overwhelming. She is approachable during office hours, which is a plus.'},
 {'professor': 'Dr. William Green',
  'subject': 'History',
  'stars': 2,
  'review': "Dr. Green's lectures are a bit dull and his grading is strict. Not the best option if you're looking for an easy class."},
 {'professor': 'Dr. Sarah Kim',
  'subject': 'Physics',
  'stars': 4,
  'review': 'Dr. 

In [5]:
import openai

processed_data = []
client = openai.OpenAI(api_key="")

# Create embeddings for each review
for review in data["reviews"]:
    response = client.embeddings.create(
        input=review['review'], model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append(
        {
            "values": embedding,
            "id": review["professor"],
            "metadata":{
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        }
    )


In [6]:
index = pc.Index("rag")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1",
)
print(f"Upserted count: {upsert_response['upserted_count']}")

# Print index statistics
print(index.describe_index_stats())


Upserted count: 30
{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 30}},
 'total_vector_count': 30}
